In [ ]:
!pip install transformers datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the combined dataset from Google Drive
df_combined = pd.read_csv('/content/drive/MyDrive/osdg_community_dataset_final.csv')

In [ ]:
# Rename the 'sdg' column to 'labels' and adjust the label values to be within the range 0-16
df_combined = df_combined.rename(columns={"sdg": "labels"})
df_combined['labels'] = df_combined['labels'] - 1  # Adjust labels to range from 0 to 16

In [ ]:
from datasets import Dataset

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_combined)

In [ ]:
from transformers import AutoTokenizer

# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    tokens = tokenizer(examples['text'], padding="max_length", truncation=True)
    tokens["labels"] = examples["labels"]
    return tokens

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/32520 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset into training and validation sets
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Remove the columns not needed for training
train_dataset = train_dataset.remove_columns(['doi', 'text_id', 'text', 'labels_negative', 'labels_positive', 'agreement'])
eval_dataset = eval_dataset.remove_columns(['doi', 'text_id', 'text', 'labels_negative', 'labels_positive', 'agreement'])

# Set the format of the datasets to PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=17)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch] accelerate -U

In [ ]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/SDGResults',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    logging_dir='/content/drive/MyDrive/SDGResultLogs',
    save_steps=500,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=17)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.846300,0.770242


Epoch,Training Loss,Validation Loss
1,0.846300,0.770242
2,0.541400,0.714975
3,0.304200,0.761548


TrainOutput(global_step=5490, training_loss=0.6136747655538479, metrics={'train_runtime': 8139.1521, 'train_samples_per_second': 10.788, 'train_steps_per_second': 0.675, 'total_flos': 2.310531448522752e+16, 'train_loss': 0.6136747655538479, 'epoch': 3.0})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Specify the path where you want to save the model and tokenizer
save_directory = '/content/drive/MyDrive/SDGFinal/'

# Load the final checkpoint
model_checkpoint = '/content/drive/MyDrive/SDGResults/checkpoint-5000'  # Adjust the path to your last checkpoint

# Load the tokenizer and model from the checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

# Save the final model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

OSError: Can't load tokenizer for '/content/drive/MyDrive/SDGResults/checkpoint-5000'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/drive/MyDrive/SDGResults/checkpoint-5000' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Specify the path where you want to save the model and tokenizer
save_directory = '/content/drive/MyDrive/SDGFinal/'

# Load the final checkpoint
model_checkpoint = '/content/drive/MyDrive/SDGResults/checkpoint-5000'  # Adjust the path to your last checkpoint

# Load the pre-trained tokenizer (from the original model used for training)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# Load the model from the checkpoint
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

# Save the final model and tokenizer
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/SDGResults/checkpoint-5000'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
from google.colab import drive
drive.mount('/content/drive')

# Verify directory content
model_path = '/content/drive/MyDrive/SDGFinal/'
if not os.path.exists(model_path):
    raise ValueError(f"Model path {model_path} does not exist.")

# List the files in the directory to ensure it's the correct path
print("Files in model directory:", os.listdir(model_path))

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()

# Function to analyze text
def analyze_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # Check for NaN values in the logits
        if torch.isnan(logits).any():
            print("Warning: NaN values detected in the logits.")
            print("Logits:", logits)
            print("Inputs:", inputs)
            return None

        predictions = torch.softmax(logits, dim=1)
        return predictions

# Example texts to classify
texts = [
    "Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development"
]

# Analyze multiple texts
for text in texts:
    print(f"Analyzing text: {text}")
    predictions = analyze_text(text)

    if predictions is not None:
        # Print the predictions
        for idx, score in enumerate(predictions[0]):
            print(f"SDG {idx + 1}: {score:.4f}")

        # Print the most likely SDG
        most_likely_sdg = torch.argmax(predictions, dim=1).item() + 1
        print(f"The most likely SDG is: SDG {most_likely_sdg}")
    else:
        print("Model output contained NaN values and could not be processed.")
    print("\n" + "-"*50 + "\n")

# Check model configuration
config = model.config
print(f"Model configuration: {config}")

# Verify model weights
for name, param in model.named_parameters():
    if torch.isnan(param).any():
        print(f"NaN values detected in parameter: {name}")
    if torch.isinf(param).any():
        print(f"Infinite values detected in parameter: {name}")


Mounted at /content/drive
Files in model directory: ['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json']
Analyzing text: Strengthen the means of implementation and revitalize the Global Partnership for Sustainable Development
SDG 1: 0.0001
SDG 2: 0.0000
SDG 3: 0.0002
SDG 4: 0.0001
SDG 5: 0.0001
SDG 6: 0.0001
SDG 7: 0.0001
SDG 8: 0.0001
SDG 9: 0.0003
SDG 10: 0.0002
SDG 11: 0.0001
SDG 12: 0.0006
SDG 13: 0.0002
SDG 14: 0.0002
SDG 15: 0.0005
SDG 16: 0.0027
SDG 17: 0.9945
The most likely SDG is: SDG 17

--------------------------------------------------

Model configuration: BertConfig {
  "_name_or_path": "/content/drive/MyDrive/SDGFinal/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os

# Specify the path where the model and tokenizer are saved
model_path = '/content/drive/MyDrive/SDGFinal/'

# Check if the model path exists
if not os.path.exists(model_path):
    raise ValueError(f"Model path {model_path} does not exist.")

# List the files in the model directory to ensure it's the correct path
print("Files in model directory:", os.listdir(model_path))

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()

# Function to analyze text
def analyze_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        # Check for NaN values in the logits
        if torch.isnan(logits).any():
            print("Warning: NaN values detected in the logits.")
            print("Logits:", logits)
            print("Inputs:", inputs)
            return None

        predictions = torch.softmax(logits, dim=1)
        return predictions

# Example texts to classify
texts = [
    "Life on land"
]

# Analyze multiple texts
for text in texts:
    print(f"Analyzing text: {text}")
    predictions = analyze_text(text)

    if predictions is not None:
        # Print the predictions
        for idx, score in enumerate(predictions[0]):
            print(f"SDG {idx + 1}: {score:.4f}")

        # Print the most likely SDG
        most_likely_sdg = torch.argmax(predictions, dim=1).item() + 1
        print(f"The most likely SDG is: SDG {most_likely_sdg}")
    else:
        print("Model output contained NaN values and could not be processed.")
    print("\n" + "-"*50 + "\n")

# Check model configuration
config = model.config
print(f"Model configuration: {config}")

# Verify model weights
for name, param in model.named_parameters():
    if torch.isnan(param).any():
        print(f"NaN values detected in parameter: {name}")
    if torch.isinf(param).any():
        print(f"Infinite values detected in parameter: {name}")


Files in model directory: ['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json']
Analyzing text: Life on land
SDG 1: 0.3116
SDG 2: 0.2849
SDG 3: 0.0268
SDG 4: 0.0066
SDG 5: 0.0624
SDG 6: 0.0333
SDG 7: 0.0102
SDG 8: 0.0062
SDG 9: 0.0075
SDG 10: 0.0130
SDG 11: 0.1360
SDG 12: 0.0022
SDG 13: 0.0070
SDG 14: 0.0110
SDG 15: 0.0781
SDG 16: 0.0026
SDG 17: 0.0008
The most likely SDG is: SDG 1

--------------------------------------------------

Model configuration: BertConfig {
  "_name_or_path": "/content/drive/MyDrive/SDGFinal/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": 